In [ ]:
import pandas as pd
import numpy as np

!pip install haversine
from haversine import haversine, Unit

In [ ]:
df=pd.read_csv('/content/India Cities LatLng.csv')

In [ ]:
df.head()

,city,latitude,longitude,state
0,Delhi,28.6600,77.2300,Delhi
1,Mumbai,18.9667,72.8333,maharashtra
2,kolkata,22.5411,88.3378,West Bengal
3,Bangalore,12.9699,77.5980,karnataka
4,Chennai,13.0825,80.2750,tamil nadu




---



---



#city and radius

---



---



In [ ]:
import folium
from haversine import haversine, Unit

def get_city_coordinates(city_name, df):
    city_data = df[df['city'].str.lower() == city_name.lower()]
    if not city_data.empty:
        return city_data.iloc[0]['latitude'], city_data.iloc[0]['longitude']
    else:
        raise ValueError(f"City '{city_name}' not found in the dataset")

def within_radius(row, reference_point, radius):
    city_coords = (row['latitude'], row['longitude'])
    distance = haversine(reference_point, city_coords, unit=Unit.KILOMETERS)
    return distance <= radius

def find_cities_within_radius(city_name, radius, df):
    try:
        reference_point = get_city_coordinates(city_name, df)
    except ValueError as e:
        return str(e)

    df['within_radius'] = df.apply(lambda row: within_radius(row, reference_point, radius), axis=1)

    cities_within_radius = df[df['within_radius']][['city', 'latitude', 'longitude']].to_dict(orient='records')

    return cities_within_radius, reference_point

def plot_map(cities_within_radius, reference_point, radius):
    m = folium.Map(location=reference_point, zoom_start=7)

    folium.Marker(
        location=reference_point,
        popup="Reference City",
        icon=folium.Icon(color="red")
    ).add_to(m)

    folium.Circle(
        radius=radius * 1000,
        location=reference_point,
        color="blue",
        fill=True,
        fill_opacity=0.1
    ).add_to(m)

    for city in cities_within_radius:
        folium.Marker(
            location=[city['latitude'], city['longitude']],
            popup=city['city'],
            icon=folium.Icon(color="green")
        ).add_to(m)

    m.save("map_with_radius.html")
    return m
''
def automated_city_search_with_map(df):
    city_name = input("Enter the city name: ")
    try:
        radius = float(input("Enter the radius in kilometers: "))
    except ValueError:
        return "Invalid radius. Please enter a numeric value."

    cities_within_radius, reference_point = find_cities_within_radius(city_name, radius, df)

    if isinstance(cities_within_radius, str):
          return cities_within_radius

    if cities_within_radius:
        print(f"Cities within {radius} km of {city_name}:")
        for city in cities_within_radius:
            print(f"- {city['city']}")

        return plot_map(cities_within_radius, reference_point, radius)
    else:
        return f"No cities found within {radius} km of {city_name}."

automated_city_search_with_map(df)


Enter the city name: pune
Enter the radius in kilometers: 400
Cities within 400.0 km of pune:
- Mumbai
- Pune
- surat
- Chinchvad
- kalyan
- Nashik
- Aurangabad
- thane
- Bhiwandi
- solapur
- Hubli
- Bhayandar
- kolapur
- Malegaon
- nanded
- Ulhasnagar
- Belgaum
- Gulbarga
- dhalia
- Jalgaon
- sangli
- Parbhani
- latur
- Ahmadnagar
- bijapur
- bidar
- Ichalkaranji
- Panaji
- daman




---



---



---



---



#multiple city and radius


In [ ]:
def get_city_coordinates(city_name, df):
    city_data = df[df['city'].str.lower() == city_name.lower()]
    if not city_data.empty:
        return city_data.iloc[0]['latitude'], city_data.iloc[0]['longitude']
    else:
        raise ValueError(f"City '{city_name}' not found in the dataset")

def within_radius(row, reference_point, radius):
    city_coords = (row['latitude'], row['longitude'])
    distance = haversine(reference_point, city_coords, unit=Unit.KILOMETERS)
    return distance <= radius

def find_cities_within_radius(city_name, radius, df):
    try:
        reference_point = get_city_coordinates(city_name, df)
    except ValueError as e:
        return str(e)

    df['within_radius'] = df.apply(lambda row: within_radius(row, reference_point, radius), axis=1)

    cities_within_radius = df[df['within_radius']][['city', 'latitude', 'longitude']].to_dict(orient='records')

    return cities_within_radius, reference_point

def plot_map(cities_within_radius, reference_points, radius):
    m = folium.Map(location=reference_points[0], zoom_start=7)

    for reference_point in reference_points:
        folium.Marker(
            location=reference_point,
            popup="Reference City",
            icon=folium.Icon(color="red")
        ).add_to(m)

        folium.Circle(
            radius=radius * 1000,
            location=reference_point,
            color="blue",
            fill=True,
            fill_opacity=0.1
        ).add_to(m)

    for city in cities_within_radius:
        folium.Marker(
            location=[city['latitude'], city['longitude']],
            popup=city['city'],
            icon=folium.Icon(color="green")
        ).add_to(m)

    m.save("map_with_radius.html")
    return m


def automated_city_search_with_map(df):
    city_names = input("Enter the city names (comma-separated): ").split(',')
    try:
        radius = float(input("Enter the radius in kilometers: "))
    except ValueError:
        return "Invalid radius. Please enter a numeric value."

    all_cities_within_radius = []
    reference_points = []
    for city_name in city_names:
        city_name = city_name.strip()
        cities_within_radius, reference_point = find_cities_within_radius(city_name, radius, df)

        if isinstance(cities_within_radius, str):
            print(cities_within_radius)
            continue

        if cities_within_radius:
            all_cities_within_radius.extend(cities_within_radius)
            reference_points.append(reference_point)
            print(f"Cities within {radius} km of {city_name}:")
            for city in cities_within_radius:
                print(f"- {city['city']}")

    if all_cities_within_radius:
        return plot_map(all_cities_within_radius, reference_points, radius)
    else:
        return "No cities found within the specified radius for the given input."


automated_city_search_with_map(df)


NameError: name 'df' is not defined